## Welcome back to Python Notebooks!

Didja miss me??

### And welcome to Week 4, Day 2 - introducing LangGraph!

In [ ]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random


In [ ]:
# Some useful constants

nouns = ["Cabbages", "Unicorns", "Toasters", "Penguins", "Bananas", "Zombies", "Rainbows", "Eels", "Pickles", "Muffins"]
adjectives = ["outrageous", "smelly", "pedantic", "existential", "moody", "sparkly", "untrustworthy", "sarcastic", "squishy", "haunted"]

In [ ]:
# Our favorite first step! Crew was doing this for us, by the way.
load_dotenv(override=True)


In [ ]:
def shout(text: Annotated[str, "something to be shouted"]) -> str:
    print(text.upper())
    return text.upper()

shout("hello")

### A word about "Annotated"

You probably know this; type hinting is a feature in Python that lets you specify the type of something:

`my_favorite_things: List`

But you may not know this:

You can also use something called "Annotated" to add extra information that somebody else might find useful:

`my_favorite_things: Annotated[List, "these are a few of mine"]`

LangGraph needs us to use this feature when we define our State object.

It wants us to tell it what function it should call to update the State with a new value.

This function is called a **reducer**.

LangGraph provides a default reducer called `add_messages` which takes care of the most common case.

And that hopefully explains why the State looks like this.




### Step 1: Define the State object

You can use any python object; but it's most common to use a TypedDict or a Pydantic BaseModel.

In [ ]:
# add_messages
# So that is it's like a function. And it's a function that you can, you can annotate with. If you want to say, hey this is the reducer I'd like you to use.
# It is a list.
# So it consists of a list of things of messages.
# And we are going to then because we're annotating it, we can provide an annotation that's ignored by
# Python, but it can be used by Landgraf.
# And that annotation is where we get to specify the reducer, the function that will be called in order
# to combine one state with another.

# All it does is it assumes this is a list.
# And if you return something with with an and items in the list.
# It just combines it with everything else in the list before it concatenates these lists together.


class State(BaseModel):
    messages: Annotated[list, add_messages]


### Step 2: Start the Graph Builder with this State class

In [ ]:
# And that is just a matter of calling this thing called state graph.
# Instantiating a state graph passing in state.
# And one thing to to get your head around here is that what I'm passing in there.
# The thing I've just highlighted. It's not an object I'm not instantiating.
# I'm not I'm not creating a state and passing that in with with messages and so on. Now I'm passing in the class.
# I'm passing in the type of thing that represents our state. That is what I'm using to create my state graph.
# And this is beginning the graph building process. This is part of the five steps before we actually run our Agentic framework.
graph_builder = StateGraph(State)

### Step 3: Create a Node

A node can be any python function.

The reducer that we set before gets automatically called to combine this response with previous responses


In [ ]:
# takes an old state and it returns a new state
def our_first_node(old_state: State) -> State:

    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    messages = [{"role": "assistant", "content": reply}]

    new_state = State(messages=messages)

    return new_state

# add it to the graph that is being build
# (node_name, function_represent_the_node)
graph_builder.add_node("first_node", our_first_node)

### Step 4: Create Edges

In [ ]:
# Signify the beginning of our workflow and the end of it.
# And so here what we say is we want an edge to take us from the start to our first node. And then we want another edge to go from the first node to the end.
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

### Step 5: Compile the Graph

In [ ]:
# Compiling the graph, our workflow
graph = graph_builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

### That's it! Showtime!

In [ ]:
# Remember gradio chat functions take the user's current input and the history of prior inputs, and it's
# meant to respond with the next output.
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input} # turn the message into a standard OpenAI format and put into a message
    messages = [message]
    state = State(messages=messages) # Create state object with that as the message
    result = graph.invoke(state) # Invoke our graph, and this is the key langgraph word invoke, So you invoke a graph in land graph with the state in order to get the result. And that's what's going to execute our graph. And what will come out will be the result.
    print("11111")
    print(result) # And we will print it and we will also return it. And that will come out of our chat function.
    print("22222")
    print(result["messages"])
    print("33333")
    print(result["messages"][-1])
    print("44444")
    print(result["messages"][1])
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

# Langgraph setup has nothing to do with LLMs specifically
# The node is just a function, in this case a silly function, but it's a function there and it's taking
# in a state and it's returning a state, and it doesn't need to have anything to do with llms.

### But why did I show you that?

To make the point that LangGraph is all about python functions - it doesn't need to involve LLMs!!

Now we'll do the 5 steps again, but in 1 shot:

In [ ]:
# Step 1: Define the State object
class State(BaseModel):
    messages: Annotated[list, add_messages]


In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)


In [ ]:
# Step 3: Create a Node
# So chat OpenAI is a construct from Lang Chain the sibling to Landgraf.
# Uh and that's what we'll be using to connect with our LLM.
# You can use any Llms.
# You could directly call the LLM yourself. 
# You could also, uh, use maybe OpenAI agents SDK.
llm = ChatOpenAI(model="gpt-4o-mini")

# takes old_state and return new_state
def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages) # takes the LLM and it invokes on that LLM, passing in masseges from old_state
    # And then for the new state, it creates a new state object which contains within it as in its messages
    # field, it contains the response.
    # And we return the new state. And we add that node called chatbot, uh, into our graph builder.
    new_state = State(messages=[response]) 
    return new_state

graph_builder.add_node("chatbot", chatbot_node)

In [ ]:
# Step 4: Create Edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### That's it! And, let's do this:

In [ ]:
# And then we put it all together in a simple gradio chat function.
# It takes an initial state, which is a state object set up with these messages like so.
# We then call graph dot invoke to actually call our graph.
# We print the result and we will also show the results back in Gradio.

# But one thing that's worth noting is that if I continue this conversation every time we are invoking
# this graph, and you will see what you have probably already suspected, which is that we're not actually
# keeping track of any history here.

# So because we've just got this simple graph that we were invoking each time, there's nothing particularly
# interesting happening here.
# And the state uh, is just, just contains that that uh, doesn't contain the history or anything.

def chat(user_input: str, history):
    initial_state = State(messages=[{"role": "user", "content": user_input}])
    result = graph.invoke(initial_state)
    print(result)
    return result['messages'][-1].content


gr.ChatInterface(chat, type="messages").launch()